Formula 1

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


# 1 Gather Data
We want to...

In [26]:
data_dir_path = "../data/"

results = pd.read_csv(data_dir_path + 'results.csv')
driver_standings = pd.read_csv(data_dir_path + 'driver_standings.csv')
constructor_standings = pd.read_csv(data_dir_path + 'constructor_standings.csv')
raceYears = pd.read_csv(data_dir_path + 'races.csv')
names = pd.read_csv(data_dir_path + 'drivers.csv')
raceYears = raceYears[["raceId", "year"]]
raceYears = raceYears[raceYears["year" ]>= 2010]

results = results[["raceId", "driverId", "constructorId", "grid", "position"]]
driver_standings = driver_standings[["raceId", "driverId", "position"]]
constructor_standings = constructor_standings[["raceId", "constructorId", "position"]]
names = names[["driverId", "driverRef"]]

driver_standings = driver_standings.rename(columns={"position": "driverStanding"})
constructor_standings = constructor_standings.rename(columns={"position": "constructorStanding"})

year_driver_standing = pd.merge(raceYears, driver_standings, on =["raceId"], how='inner')

results_driver_standings = pd.merge(results, year_driver_standing, on=["raceId", "driverId"], how="inner")

joined_data = pd.merge(results_driver_standings, constructor_standings, on=["raceId", "constructorId"], how="inner")

joined_data = pd.merge(joined_data, names, on = ["driverId"], how = "inner")


joined_data.head()


KeyError: "None of [Index(['driverId, driverRef'], dtype='object')] are in the [columns]"